<a href="https://colab.research.google.com/github/AryanWadhwa05/Stock-Analysis-and-Prediction/blob/main/Streamlit_Date_Features(DND).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Run this cell to load some data
!pip install streamlit
!pip install pyngrok 
!pip install yfinance
!pip install ipykernel>=5.1.2
!pip install pydeck
!pip install vaderSentiment
!pip install langdetect
!pip install emoji
!pip install praw

import tensorflow as tf
import pandas as pd
import cv2
import plotly.express as px
import numpy as np
import yfinance as yf
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import requests
import os
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect
import emoji
import praw
import statistics as stats
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

import streamlit as st



     |████████████████████████████████| 9.3 MB 11.3 MB/s 
     |████████████████████████████████| 4.3 MB 51.2 MB/s 
     |████████████████████████████████| 180 kB 41.0 MB/s 
     |████████████████████████████████| 111 kB 26.7 MB/s 
     |████████████████████████████████| 164 kB 71.6 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 127 kB 65.3 MB/s 
     |████████████████████████████████| 792 kB 53.9 MB/s 
     |████████████████████████████████| 374 kB 60.4 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=299f04f24d2336eca4e08f59612ac2d8f3108866662dcf9e3510778ad8f47714
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      

     |████████████████████████████████| 745 kB 14.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=9aba10ae135ba167954dc15af31eaa03a1d7d197d5ae5ff0af6f593b1b0248f2
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 6.4 MB 34.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ip

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



FileNotFoundError: ignored

In [14]:
%%writefile app.py

# All imports
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import praw
import requests
import os
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect
import emoji
import statistics as stats
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

# Streamlit title
st.title("Stock Market Analysis")

# Streamlit Sidebar
st.sidebar.title("Stocks")

stock_search = st.sidebar.text_input('Stock Search')

keywords_for_NLP = st.sidebar.text_input('NLP keywords')

# Checkboxes

nlp_model_checkbox = st.sidebar.checkbox("Press this if you want to enable NLP model")

lstm_model_checkbox = st.sidebar.checkbox("Press this if you want to enable LSTM model")

arima_model_checkbox = st.sidebar.checkbox("Press this if you want to enable ARIMA model")

ready_to_run_code = st.checkbox("Click this when you are ready to run the code")

# Pick variables such as stock ticker and what we want to train on (Close, High, Open)

# Pick prediction date
prediction_date = st.sidebar.date_input('Prediction Date')

# Ticker string if multiple tickers
yf.pdr_override()
tickers = ""

# setting up reddit variables/dependencies
reddit_id = "wpotSHZd6nAbXpA-aqVTwQ"
reddit_secret = "clfa5nAXLOr3iHpbwm9RGrG7eFJZvg"
reddit_user = "HolyShoter"
reddit_pass = "GARgar109$"

reddit = praw.Reddit(
    client_id=reddit_id,
    client_secret=reddit_secret,
    password=reddit_pass,
    user_agent="USERAGENT",
    username=reddit_user,
    check_for_async = False
)

# dictionary to update VADER Lexicon
new_words = {
    'citron': -4.0,  
    'hindenburg': -4.0,        
    'moon': 4.0,
    'highs': 2.0,
    'mooning': 4.0,
    'long': 2.0,
    'short': -2.0,
    'call': 4.0,
    'calls': 4.0,    
    'put': -4.0,
    'puts': -4.0,    
    'break': 2.0,
    'tendie': 2.0,
     'tendies': 2.0,
     'town': 2.0,     
     'overvalued': -3.0,
     'undervalued': 3.0,
     'buy': 4.0,
     'sell': -4.0,
     'gone': -1.0,
     'gtfo': -1.7,
     'paper': -1.7,
     'bullish': 3.7,
     'bearish': -3.7,
     'bagholder': -1.7,
     'stonk': 1.9,
     'green': 1.9,
     'money': 1.2,
     'print': 2.2,
     'rocket': 2.2,
     'bull': 2.9,
     'bear': -2.9,
     'pumping': -1.0,
     'sus': -3.0,
     'offering': -2.3,
     'rip': -4.0,
     'downgrade': -3.0,
     'upgrade': 3.0,     
     'maintain': 1.0,          
     'pump': 1.9,
     'hot': 1.5,
     'drop': -2.5,
     'rebound': 1.5,  
     'crack': 2.5,
     "up":2.7,
     "down":-2.7,
     "comeback":3,
     "fall":-3,
     "asshole":-4,
     "suckers":-4,
     "fault":-3,
     "doesn't care":-4,
     }

# Setting up VADER

analyzer = SentimentIntensityAnalyzer()
analyzer.lexicon.update(new_words)

tweet_num = 500
reddit_num = 70

if nlp_model_checkbox:
  tweet_num = st.sidebar.number_input('Tweet num',min_value = 400,max_value = 900, value = 500)
  reddit_num = st.sidebar.number_input('Reddit num',min_value = 50,max_value = 600,value = 70)
  
  st.write("Extended sidebar, please scroll down")

if lstm_model_checkbox or arima_model_checkbox:
  display_cols = st.sidebar.multiselect('Display Data', ["Close","Low","Open","High",],default = "High")

# Download data from yahoo finance
def download_data(ticker = tickers):
  data = yf.download(ticker)
  # st.write(ticker)
  cf = data

  cf["Date"] = cf.index

  df = cf

  dataframe = data[display_cols]
  dataframe.reset_index(level=0, inplace=True)

  # Plot stock data without ml or nlp
  plt.plot(dataframe["Date"],dataframe[display_cols])
  plt.ylabel("Dollars")
  plt.xlabel("Date")
  plt.legend()
  plt.draw()

  st.pyplot(plt)

# Machine Learning starts
  cf=dataframe
  pf = cf


  # if cf.shape[0] > 5000:
  #   st.write("Dataset too big, please use more recent stock")
  #   quit()

  # LSTM dataset code
  cf.index=cf['Date']

  df['Date'] = pd.to_datetime(df['Date'])
  df['Date'] = df['Date'].dt.strftime('%d.%m.%Y')
  df['year'] = pd.DatetimeIndex(df['Date']).year
  df['month'] = pd.DatetimeIndex(df['Date']).month
  df['day'] = pd.DatetimeIndex(df['Date']).day
  df['dayofyear'] = pd.DatetimeIndex(df['Date']).dayofyear
  df['weekofyear'] = pd.DatetimeIndex(df['Date']).weekofyear
  df['weekday'] = pd.DatetimeIndex(df['Date']).weekday
  df['quarter'] = pd.DatetimeIndex(df['Date']).quarter
  df['is_month_start'] = pd.DatetimeIndex(df['Date']).is_month_start
  df['is_month_end'] = pd.DatetimeIndex(df['Date']).is_month_end

  df.loc[df['is_month_start']==False,'is_month_start']=0
  df.loc[df['is_month_start']==True,'is_month_start']=1
  df.loc[df['is_month_end']==False,'is_month_end']=0
  df.loc[df['is_month_end']==True,'is_month_end']=1

  df = df.drop(['Date','Open','Low','Close','Adj Close','Volume'], axis = 1) 


  # *********************** splitting the dataset

  X = df.drop('High', axis=1)
  y = df['High']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

  # ********************** Feature Scaling
  scaler=MinMaxScaler(feature_range=(0,1))
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.fit_transform(X_test)

  # LSTM architecture

  def lstm_architecture():
    lstm_model=Sequential()
    lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(1))

    lstm_model.compile(loss='mean_squared_error',optimizer='adam')
    lstm_model.fit(X_train,y_train,epochs=150,batch_size=25,verbose=2)

    # Testing
    predicted_closing_price=lstm_model.predict(X_test)

    valid_data=pd.DataFrame(index=range(0,len(y_test)),columns=['Date','y_test',"predicted"])

    valid_data['y_test'] = list(y_test)
    valid_data['predicted'] = predicted_closing_price
    valid_data['Date'] = y_test.index
    valid_data = valid_data.set_index('Date')
    valid_data = valid_data.sort_index()


  # st.write(valid_data.head())
  # Visuliazation LSTM
    plt.plot(valid_data)
    st.pyplot(plt)

  # Display LSTM prediction datatset
    # st.write(valid_data)

    #  ***************************************************** Prediction tools
    def lstm_prediction(prediction_date = prediction_date):
      data = [prediction_date]
      value_data=pd.DataFrame(data,index=range(0,1),columns=['Date'])

      value_data['year'] = pd.DatetimeIndex(value_data['Date']).year
      value_data['month'] = pd.DatetimeIndex(value_data['Date']).month
      value_data['day'] = pd.DatetimeIndex(value_data['Date']).day
      value_data['dayofyear'] = pd.DatetimeIndex(value_data['Date']).dayofyear
      value_data['weekofyear'] = pd.DatetimeIndex(value_data['Date']).weekofyear
      value_data['weekday'] = pd.DatetimeIndex(value_data['Date']).weekday
      value_data['quarter'] = pd.DatetimeIndex(value_data['Date']).quarter
      value_data['is_month_start'] = pd.DatetimeIndex(value_data['Date']).is_month_start
      value_data['is_month_end'] = pd.DatetimeIndex(value_data['Date']).is_month_end

      value_data.loc[value_data['is_month_start']==False,'is_month_start']=0
      value_data.loc[value_data['is_month_start']==True,'is_month_start']=1
      value_data.loc[value_data['is_month_end']==False,'is_month_end']=0
      value_data.loc[value_data['is_month_end']==True,'is_month_end']=1

      value_data = value_data.drop(['Date'], axis = 1) 

      value_data=scaler.transform(value_data)
      pr = lstm_model.predict(value_data)
      st.write(pr)

    lstm_prediction()
  
  if lstm_model_checkbox:
    lstm_architecture()


# NLP
def reddit_twitter_webscraping_sentiment(twitter_num_of_tweets = tweet_num,reddit_num_of_comment = reddit_num):
  bearer_token = "AAAAAAAAAAAAAAAAAAAAAGl9WAEAAAAAup10asDXc%2BPNXX7yTpSv%2B%2B0nJQ0%3DU2xJJZhYx8LHXFfEm4NqzTNHtrCRjRufnTjw9fZcdggkQGDl0O"

  def replace_RT(tweet, default_replace=""):
    tweet = re.sub('RT\s+', default_replace, tweet)
    return tweet

  def replace_user(tweet, default_replace=""):
    tweet = re.sub('\B@\w+', default_replace, tweet)
    return tweet

  def replace_url(tweet, default_replace=""):
    tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
    return tweet

  def bearer_oauth(r):
      """
      Method required by bearer token authentication.
      """

      r.headers["Authorization"] = f"Bearer {bearer_token}"
      r.headers["User-Agent"] = "v2FilteredStreamPython"
      return r


  def get_rules():
      response = requests.get(
          "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
      )
      if response.status_code != 200:
          raise Exception(
              "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
          )
      print(json.dumps(response.json()))
      return response.json()


  def delete_all_rules(rules):
      if rules is None or "data" not in rules:
          return None

      ids = list(map(lambda rule: rule["id"], rules["data"]))
      payload = {"delete": {"ids": ids}}
      response = requests.post(
          "https://api.twitter.com/2/tweets/search/stream/rules",
          auth=bearer_oauth,
          json=payload
      )
      if response.status_code != 200:
          raise Exception(
              "Cannot delete rules (HTTP {}): {}".format(
                  response.status_code, response.text
              )
          )
      print(json.dumps(response.json()))


  def set_rules(delete):
      # You can adjust the rules if needed
      sample_rules = [
          {"value": "TSLA", "tag": "TSLA"},
          {"value": "Elon", "tag": "Elon"},
          {"value": "Musk", "tag": "Musk"},
          {"value": "Tesla", "tag": "Tesla"},
      ]
      payload = {"add": sample_rules}
      response = requests.post(
          "https://api.twitter.com/2/tweets/search/stream/rules",
          auth=bearer_oauth,
          json=payload,
      )
      if response.status_code != 201:
          raise Exception(
              "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
          )
      print(json.dumps(response.json()))


  def get_stream(set):
      response = requests.get(
          "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
      )
      print(response.status_code)
      if response.status_code != 200:
          raise Exception(
              "Cannot get stream (HTTP {}): {}".format(
                  response.status_code, response.text
              )
          )
      lst = []
      KEYWORDS = ["tesla","tsla","elon","musk"]
      def reddit_stream(KEYWORDS = KEYWORDS,amount_of_reddit = reddit_num_of_comment):
        for i in range(len(KEYWORDS)):
            try:
              for comment in reddit.subreddit(KEYWORDS[i]).stream.comments():
                sentence = comment.body
                display_sentence = comment.body
                sentence = replace_RT(sentence) # replace retweet
                sentence = replace_user(sentence) # replace user tag
                sentence = replace_url(sentence) # replace url
                sentence = sentence.lower()
                sentiment_reddit = analyzer.polarity_scores(sentence)
                if len(lst) < amount_of_reddit*(i+1):
                  print("NEW " + KEYWORDS[i] + ": " + display_sentence + " " + str(sentiment_reddit["compound"]))
                  lst.append(sentiment_reddit["compound"])
                else:
                  break
            except:
              continue
        # print(len(lst))
      reddit_stream()
  # and len(lst) <= len(KEYWORDS)*40+200
      lst_twitter = []
      def twitter_streams(amount_tweets = twitter_num_of_tweets):
        for response_line in response.iter_lines():
            if response_line:
              json_response = json.loads(response_line)
              tweet = json_response["data"]["text"]
              tweet = replace_RT(tweet) # replace retweet
              tweet = replace_user(tweet) # replace user tag
              tweet = replace_url(tweet) # replace url
              tweet = tweet.lower()
              sentiment = analyzer.polarity_scores(tweet)
        
              try:
                if detect(tweet) == "en" and len(lst_twitter) <= amount_tweets:
                  print("New Twitter: " + tweet + " " + str(sentiment["compound"]))
                  lst_twitter.append(sentiment["compound"])
                  # print(len(lst_twitter))
                elif len(lst_twitter) > amount_tweets:
                  break
                else:
                  pass
              except:
                pass
      twitter_streams()
        
      def web_scraping_sentiment():
        finviz_url = "https://finviz.com/quote.ashx?t="
        apewisdom_url = "https://apewisdom.io/stocks/"
        ticker = "TSLA"
        finviz_url = finviz_url + ticker
        apewisdom_url = apewisdom_url + ticker + "/"

        finviz_req = Request(url=finviz_url, headers={'user-agent': 'my-app/0.0.1'})
        finviz_response = urlopen(finviz_req)
        apewisdom_req = Request(url=apewisdom_url, headers = {"user-agent": "my-app/0.0.1"})
        apewisdom_response = urlopen(apewisdom_req)
        df = pd.DataFrame(columns=['News_Title', 'Time'])
        sentiment_list = []
        news_table = {}


        soup = BeautifulSoup(apewisdom_response)
        html = BeautifulSoup(finviz_response)


        mentioning_users = soup.findAll("div",{"class":"details-small-tile"})[-2]
        upvotes = soup.findAll("div",{"class":"details-small-tile"})[-3]
        mentions = soup.findAll("div",{"class":"details-small-tile"})[-4]
        news_table = html.find(id='news-table')

        mentioning_users_percentage = mentioning_users.find("span").text
        upvotes_percentage = upvotes.find("span").text
        mentions_percentage = mentions.find("span").text
        sentiment = soup.findAll("div",{"class":"tile-value"})[-1].text


        dataRows = news_table.findAll('tr')


        for i, table_row in enumerate(dataRows):
            a_text = table_row.a.text
            td_text = table_row.td.text
            
            df = df.append({'News_Title': a_text, 'Time': td_text}, ignore_index=True)


        for i in range(50):
          word = df["News_Title"][i]
          news_title_sentiment = analyzer.polarity_scores(word)
          news_title_sentiment = news_title_sentiment["compound"]
          sentiment_list.append(news_title_sentiment)

        sentiment_list = stats.mean(sentiment_list)


        sentiment = sentiment[0:2]
        sentiment = int(sentiment) - 50
        if sentiment > 1:
          sentiment = sentiment*2/100
        elif sentiment < -1:
          sentiment = sentiment*2/100

        global final_sentiment
        final_sentiment = sentiment + sentiment_list
        # print(final_sentiment)
      web_scraping_sentiment()



      def find_mean_from_list():
        global final_list
        final_list = []
        final_list = lst + lst_twitter
        # print(len(final_list))
        final_list = stats.mean(final_list)
        # print("The mean is: " + str(final_list))
      find_mean_from_list()

      very_last_sentiment = (final_list + final_sentiment)/2
      st.write("The final sentiment is: " + str(very_last_sentiment))

  def main():
      rules = get_rules()
      delete = delete_all_rules(rules)
      set = set_rules(delete)
      get_stream(set)


  if __name__ == "__main__":
      main()


def main2():
  if lstm_model_checkbox or arima_checkbox:
    tickers = stock_search.replace(","," ")
    if len(tickers) > 0:
      download_data(tickers)
    else:
      st.write("Please type in a ticker")
    if nlp_model_checkbox:
      if len(keywords_for_NLP) > 0:
        reddit_twitter_webscraping_sentiment()
  elif nlp_model_checkbox:
    if len(keywords_for_NLP) > 0:
      reddit_twitter_webscraping_sentiment()
  else:
    st.write("Please check one checkbox")

if ready_to_run_code:
  main2()

Overwriting app.py


In [3]:
!ngrok authtoken 23D1NySpe3UL1BoP2zl0GzEhkIw_7ZZp4VJS5pZi9snaZSM2K

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py

2022-01-24 03:22:55.473 INFO    pyngrok.ngrok: Opening tunnel named: http-80-5fca895a-bbb4-44cd-bb72-1e8faad2f191
2022-01-24 03:22:55.507 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="no configuration paths supplied"
2022-01-24 03:22:55.513 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-01-24 03:22:55.515 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-01-24 03:22:55.521 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2022-01-24 03:22:55.846 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2022-01-24 03:22:55.848 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:55+0000 lvl=info msg="client session established" obj=csess id=9360232a55dc
2

NgrokTunnel: "http://3dc1-35-204-11-63.ngrok.io" -> "http://localhost:80"


2022-01-24 03:22:58.232 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=09bc5c7f9484 l=127.0.0.1:80 r=24.188.197.152:58893



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.204.11.63:80



2022-01-24 03:22:58.458 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=fec9624f0612 l=127.0.0.1:80 r=24.188.197.152:58893
2022-01-24 03:22:58.488 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=41ebe38d4c3d l=127.0.0.1:80 r=24.188.197.152:58897
2022-01-24 03:22:58.500 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=fa96611d7cbb l=127.0.0.1:80 r=24.188.197.152:58896
2022-01-24 03:22:58.503 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=8808ce4f31ba l=127.0.0.1:80 r=24.188.197.152:58898
2022-01-24 03:22:58.516 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:58+0000 lvl=info msg="join connections" obj=join id=4706fc8df6c4 l=127.0.0.1:80 r=24.188.197.152:58899
2022-01-24 03:22:59.126 INFO    pyngrok.process.ngrok: t=2022-01-24T03:22:59+0000 lvl=info msg="join connections" o

/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



2022-01-24 03:23:02.105 INFO    pyngrok.process.ngrok: t=2022-01-24T03:23:02+0000 lvl=info msg="join connections" obj=join id=bd7f0b544ab0 l=127.0.0.1:80 r=24.188.197.152:58902
2022-01-24 03:23:02.713 INFO    pyngrok.process.ngrok: t=2022-01-24T03:23:02+0000 lvl=info msg="join connections" obj=join id=8bd14278c821 l=127.0.0.1:80 r=24.188.197.152:58898
2022-01-24 03:23:02.715 INFO    pyngrok.process.ngrok: t=2022-01-24T03:23:02+0000 lvl=info msg="join connections" obj=join id=ae2b8c3d1b6a l=127.0.0.1:80 r=24.188.197.152:58894
2022-01-24 03:23:02.718 INFO    pyngrok.process.ngrok: t=2022-01-24T03:23:02+0000 lvl=info msg="join connections" obj=join id=a5e0d1e42b65 l=127.0.0.1:80 r=24.188.197.152:58893
2022-01-24 03:23:02.721 INFO    pyngrok.process.ngrok: t=2022-01-24T03:23:02+0000 lvl=info msg="join connections" obj=join id=788903c40efd l=127.0.0.1:80 r=24.188.197.152:58896


/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

2022-01-24 03:23:10.945 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
2022-01-24 03:23:11.021 No handles with